# Qaror Daraxti

---

- **Source:** https://medium.com/@penggongting/implementing-decision-tree-from-scratch-in-python-c732e7c69aea
- Iris Dataset: https://scikit-learn.org/stable/auto\_examples/datasets/plot\_iris\_dataset.html

In [1]:
from pprint import pprint
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
def entropy_cal(c1, c2):
    """
    Returns entropy of a group of data
    c1: count of one class
    c2: count of another class
    """

    if c1==0 or c2==0:  # when there is one class in the group, entropy is 0
        return 0
    n = c1 + c2
    return -(c1/n)*np.log2(c1/n) - (c2/n)*np.log2(c2/n)

In [3]:
# Get the entropy of one big circle showing above
def entropy_of_one_division(division):
    """
    Returns entropy of a divided group of data
    Data may have multiple classes
    """

    s = 0
    n = len(division)
    classes = set(division)
    for c in classes:  # for each class, get entropy
        n_c = sum(division==c)
        e = n_c/n * entropy_cal(sum(division==c), sum(division!=c))  # weighted avg
        s += e
    return s, n

In [4]:
# The whole entropy of two big circles combined
def get_entropy(y_predict, y_real):
    """
    Returns entropy of a split
    y_predict is the split decision, True/Fasle, and y_true can be multi class
    """
    if len(y_predict) != len(y_real):
        print('They have to be the same length')
        return None
    n = len(y_real)
    s_true, n_true = entropy_of_one_division(y_real[y_predict]) # left hand side entropy
    s_false, n_false = entropy_of_one_division(y_real[~y_predict]) # right hand side entropy
    s = n_true/n * s_true + n_false/n * s_false # overall entropy, again weighted average
    return s

In [5]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
    
    def fit(self, x, y, par_node={}, depth=0):
        if par_node is None: 
            return None
        elif len(y) == 0:
            return None
        elif self.all_same(y):
            return {'val':y[0]}
        elif depth >= self.max_depth: 
            return None
        else: 
            col, cutoff, entropy = self.find_best_split_of_all(x, y)    # find one split given an information gain 
            y_left = y[x[:, col] < cutoff]
            y_right = y[x[:, col] >= cutoff]
            par_node = {'col': iris.feature_names[col], 'index_col':col,
                        'cutoff':cutoff,
                       'val': np.round(np.mean(y))}
            par_node['left'] = self.fit(x[x[:, col] < cutoff], y_left, {}, depth+1)
            par_node['right'] = self.fit(x[x[:, col] >= cutoff], y_right, {}, depth+1)
            self.depth += 1 
            self.trees = par_node
            return par_node
    
    def find_best_split_of_all(self, x, y):
        col = None
        min_entropy = 1
        cutoff = None
        for i, c in enumerate(x.T):
            entropy, cur_cutoff = self.find_best_split(c, y)
            if entropy == 0:    # find the first perfect cutoff. Stop Iterating
                return i, cur_cutoff, entropy
            elif entropy <= min_entropy:
                min_entropy = entropy
                col = i
                cutoff = cur_cutoff
        return col, cutoff, min_entropy
    
    def find_best_split(self, col, y):
        min_entropy = 10
        n = len(y)
        for value in set(col):
            y_predict = col < value
            my_entropy = get_entropy(y_predict, y)
            if my_entropy <= min_entropy:
                min_entropy = my_entropy
                cutoff = value
        return min_entropy, cutoff
    
    def all_same(self, items):
        return all(x == items[0] for x in items)
                                           
    def predict(self, x):
        tree = self.trees
        results = np.array([0]*len(x))
        for i, c in enumerate(x):
            results[i] = self._get_prediction(c)
        return results
    
    def _get_prediction(self, row):
        cur_layer = self.trees
        while cur_layer.get('cutoff'):
            if row[cur_layer['index_col']] < cur_layer['cutoff']:
                cur_layer = cur_layer['left']
            else:
                cur_layer = cur_layer['right']
        else:
            return cur_layer.get('val')

In [6]:
iris = load_iris()

x = iris.data
y = iris.target

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((112, 4), (38, 4), (112,), (38,))

In [8]:
qaror_daraxti = DecisionTreeClassifier(max_depth=7)
mashq_jarayoni = qaror_daraxti.fit(x_train, y_train)
pprint(mashq_jarayoni)

{'col': 'petal width (cm)',
 'cutoff': 1.0,
 'index_col': 3,
 'left': {'val': 0},
 'right': {'col': 'petal width (cm)',
           'cutoff': 1.8,
           'index_col': 3,
           'left': {'col': 'petal length (cm)',
                    'cutoff': 5.0,
                    'index_col': 2,
                    'left': {'val': 1},
                    'right': {'col': 'petal width (cm)',
                              'cutoff': 1.6,
                              'index_col': 3,
                              'left': {'val': 2},
                              'right': {'val': 1},
                              'val': 2.0},
                    'val': 1.0},
           'right': {'val': 2},
           'val': 1.0},
 'val': 1.0}


In [9]:
predicted_y_test = qaror_daraxti.predict(x_test)
predicted_y_test

array([1, 0, 2, 2, 2, 0, 0, 0, 1, 0, 0, 1, 2, 2, 0, 0, 2, 2, 2, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 2, 2, 0, 1, 2, 2, 1, 2, 1, 0])

In [10]:
print(classification_report(y_test, predicted_y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.83      0.91      0.87        11
           2       0.92      0.86      0.89        14

    accuracy                           0.92        38
   macro avg       0.92      0.92      0.92        38
weighted avg       0.92      0.92      0.92        38

